In [2]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from IPython.display import FileLink
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [1]:
#deldata and update after processing [FINISH LATER] -> done

In [ ]:
delData = openpyxl.load_workbook('PO Lazada x ULI Template Processing.xlsx')

In [ ]:
sheetPH = delData['Product Hierarchy']
delData.remove(sheetPH)

In [ ]:
sheetPO = delData['SKU List of Purchase Order For ']
delData.remove(sheetPO)

In [ ]:
sheetPricelistLAZ = delData['Updated Pricelist (LAZ SKU UPDA']
delData.remove(sheetPricelistLAZ)

In [ ]:
sheetPC = delData['PC Mapping']
delData.remove(sheetPC)

In [ ]:
sheetStock = delData['ANDROMEDA']
delData.remove(sheetStock)

In [ ]:
sheetProses = delData['proses']
delData.remove(sheetProses)

In [ ]:
delData.save('PO Lazada x ULI Template Processing.xlsx')

In [ ]:
#define master data (PRICELIST ULI & PRODUCT HIERARCHY)

In [ ]:
#define PO Detail LAZADA

In [ ]:
POdetail = pd.read_excel('Unilever - Lazada Weekly Sell Out 2022-2023_RAW.xlsx', sheet_name='Sheet1')

In [ ]:
POdetail.head()

In [ ]:
POdetail = POdetail.drop(columns=['pc1'])

In [ ]:
POdetail.head()

In [ ]:
with pd.ExcelWriter('PO Lazada x ULI Template Processing.xlsx', mode='a', engine='openpyxl') as writer:
    POdetail.to_excel(writer, sheet_name='SKU List of Purchase Order For ', index=False)

In [ ]:
#define laz ecatalogue

In [ ]:
Laz_ecatalog = pd.read_excel('Lazada Catalog.xlsx', sheet_name='Sheet1')

In [ ]:
Laz_ecatalog.head()

In [ ]:
with pd.ExcelWriter('PO Lazada x ULI Template Processing.xlsx', mode='a', engine='openpyxl') as writer:
    Laz_ecatalog.to_excel(writer, sheet_name='Updated Pricelist (LAZ SKU UPDA', index=False)

In [ ]:
ProductHierarchy = pd.read_excel('Product Hierarchy.xlsx', sheet_name='ID PH')

In [ ]:
ProductHierarchy.head()

In [ ]:
with pd.ExcelWriter('PO Lazada x ULI Template Processing.xlsx', mode='a', engine='openpyxl') as writer:
    ProductHierarchy.to_excel(writer, sheet_name='Product Hierarchy', index=False)

In [ ]:
file_processing = 'PO Lazada x ULI Template Processing.xlsx'

In [ ]:
df = pd.read_excel(file_processing, sheet_name='SKU List of Purchase Order For ')

In [ ]:
#merger & pisah per bundle

In [ ]:
df_PC = pd.read_excel(file_processing, sheet_name='Updated Pricelist (LAZ SKU UPDA', usecols=['SKU', 'PC Code', 'Multiplier', 'pc1', 'qty1', 'pc2', 'qty2', 'pc3', 'qty3', 'pc4', 'qty4'])

In [ ]:
df = pd.merge(left=df, right=df_PC, how='left', left_on='SKU', right_on='SKU')

In [ ]:
df.head()

In [ ]:
total_rows = df.shape[0]
print(total_rows)

In [ ]:
#!!!!!

In [ ]:
if df['PC Code'].astype(str).str.contains('-', na=False).any():
    # Membuat DataFrame baru untuk data bundle
    df_bundle = df[df['PC Code'].astype(str).str.contains('-', na=False)].copy()
    df_bundle['PC Code'] = df_bundle['PC Code'].astype(str)

    
    
    # Delete data bundle dari DataFrame asli
    df = df[~df['PC Code'].astype(str).str.contains('-', na=False)].copy()

In [ ]:
#pisahkan bundle ke baris baru untuk masing masing PC code -> DONE

In [ ]:
df.dtypes

In [ ]:
if 'df_bundle' in locals():
    df_bundle['PC Code'] = df_bundle['PC Code'].astype(str)

In [ ]:
if 'df_bundle' in locals():
    df_bundle['PC Code Original'] = df_bundle['PC Code']

In [ ]:
if 'df_bundle' in locals():
    df_bundle = (df_bundle.assign(PC_Code=df_bundle['PC Code'].str.split('-'))
                 .explode('PC Code')
                 .reset_index(drop=True))

In [ ]:
df_bundle.head()

In [ ]:
if 'df_bundle' in locals():
    df_bundle = (df_bundle.set_index(['DATE', 'SKU', 'PRODUCT NAME', 'BRAND', 'GROUP COMPANY', 'COMMERCIAL CATEGORY', 'ITEM SOLD', 'YEAR', 'SKU UNILEVER CODE', 'pc1', 'qty1', 'pc2', 'qty2', 'pc3', 'qty3', 'pc4', 'qty4', 'Multiplier', 'PC Code Original'], append=True)
                 ['PC Code']
                 .str.split('-', expand=True)
                 .stack()
                 .reset_index(level=0, drop=True)
                 .reset_index(name='PC Code'))

In [ ]:
#mapping pc on qty then input on multiplier

In [ ]:
if 'df_bundle' in locals():
    # Create an empty dictionary to store the mapping
    mapping_dict = {}

    # Iterate over each row in the DataFrame
    for index, row in df_bundle.iterrows():
        pc_code = row['PC Code']
        qty_columns = ['qty1', 'qty2', 'qty3', 'qty4']
        pc_columns = ['pc1', 'pc2', 'pc3', 'pc4']

        # Iterate over pc_columns and qty_columns simultaneously
        for pc_col, qty_col in zip(pc_columns, qty_columns):
            if pc_col in row and not pd.isna(row[pc_col]):
                sku_pc_code = f"{row[pc_col]}-{row['PC Code']}"
                if sku_pc_code not in mapping_dict:
                    mapping_dict[sku_pc_code] = row[qty_col]
                else:
                    mapping_dict[sku_pc_code] = max(mapping_dict[sku_pc_code], row[qty_col])

    # Update the Multiplier column based on the mapping
    df_bundle['Multiplier'] = df_bundle.apply(lambda row: mapping_dict.get(f"{row['pc1']}-{row['PC Code']}"), axis=1)


In [ ]:
if 'df_bundle' in locals():
    display(df_bundle)
    
    #MULTIPLIER JADI 2?

In [ ]:
if 'df_bundle' in locals():
    df = pd.concat([df, df_bundle], ignore_index=True)

In [ ]:
delisted_data = df.loc[df['PC Code'] == 'DELISTED', 'PC Code']
delisted_data

In [ ]:
def replace_delisted_with_pc1(row):
    if row['PC Code'] == 'DELISTED':
        return row['pc1']
    else:
        return row['PC Code']

df['PC Code'] = df.apply(replace_delisted_with_pc1, axis=1)

In [ ]:
if all(col in df.columns for col in ['pc1', 'qty1', 'pc2', 'qty2', 'pc3', 'qty3', 'pc4', 'qty4']):
    df = df.drop(['pc1', 'qty1', 'pc2', 'qty2', 'pc3', 'qty3', 'pc4', 'qty4'], axis=1)

df.head()

In [ ]:
#column final qty

In [ ]:
df['Multiplier'] = pd.to_numeric(df['Multiplier'], errors='coerce')

In [ ]:
df.dtypes

In [ ]:
df['Final Order Qty (pcs)'] = df['ITEM SOLD'] * df['Multiplier']

In [ ]:
df.head()

In [ ]:
df_ProductHierarchy = pd.read_excel(file_processing, sheet_name='Product Hierarchy', usecols=['ULI Product Hierarchy', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 46', 'Unnamed: 50'])

In [ ]:
df_ProductHierarchy.head()

In [ ]:
df_ProductHierarchy.dtypes

In [ ]:
df['PC Code'] = df['PC Code'].astype(str)
df_ProductHierarchy['ULI Product Hierarchy'] = df_ProductHierarchy['ULI Product Hierarchy'].astype(str)

In [ ]:
df = pd.merge(left=df, right=df_ProductHierarchy, how='left', left_on='PC Code', right_on='ULI Product Hierarchy')

In [ ]:
df.head(100)

In [ ]:
df = df.rename(columns={'Unnamed: 1_y': 'SKU Unilever Description', 'Unnamed: 3_y': 'PC/Cs', 'Unnamed: 46_y': 'Category', 'Unnamed: 50_y': 'Division' })

In [ ]:
df = df.drop(columns=['ULI Product Hierarchy_x', 'Unnamed: 1_x', 'Unnamed: 3_x', 'Unnamed: 46_x', 'Unnamed: 50_x', 'ULI Product Hierarchy_y'])

In [ ]:
df.dtypes

In [ ]:
df['PC/Cs'] = df['PC/Cs'].astype(float)

In [ ]:
df['Final Qty in CS'] = df.apply(lambda row: row['Final Order Qty (pcs)'] / row['PC/Cs'], axis=1)

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['SKU UNILEVER CODE'])

In [ ]:
df.to_excel('24D3_Lazada Weekly Sell Out.xlsx', sheet_name='Result', index=False)

In [ ]:
FileLink('24D3_Lazada Weekly Sell Out.xlsx')